In [10]:
import numpy as np
import dask_image.imread
import cc3d # https://github.com/seung-lab/connected-components-3d
from skimage import io, morphology, measure
from project_module import *
from Neuron_analysis import *
from anne_module import *

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%load_ext Cython

In [4]:
%%cython
import numpy as np
cimport numpy as np

def set_to_int(np.ndarray[float, ndim=3] vol, np.ndarray[long long, ndim=2] list_coords, int value = 0):
    cdef int i = 0
    for i in range(len(list_coords)):
        vol[list_coords[i][0],list_coords[i][1],list_coords[i][2]] = value
    
    return vol

In [5]:
def trim_small_elements(stack_filename, min_threshold, max_threshold=-1, keep_label=False):
    # Removing small elements increases the mean probability of remaining pixels. (Post-skeletonization)
    # Small elements should be associated with lower probabilities of axon presence (artifacts, blood vessels,...)
    # We want the "big" stuff
    
    label_stack = from_folder_to_stack(stack_filename) # loading stack a first time for connected-components labeling
    label_stack = label_stack.view(dtype=np.float32) # forces a specific size (should be native size after segmentation) (check if worth doing)
    label_stack[label_stack > 0] = 1 # binarization, checking below 0 should be unnecessary (if proper workflow order)
    label_stack, N = cc3d.connected_components(label_stack, out_dtype=np.uint32, return_N=True) # better than skimage ?
    #label_stack = measure.label(label_stack, background=0) #computationally expensive
    print('Start: individual "blobs":', N)
    
    propsa = measure.regionprops(label_stack)
    del label_stack #liberate memory
    
    index_list = []
    kept_labels = []
    for count, item in enumerate(propsa):
        if len(item.coords) < min_threshold:
            index_list.append(count)
        elif max_threshold > 0 and len(item.coords) > max_threshold:
            index_list.append(count)
        else:
            kept_labels.append(count)
            
    
    #del propsa #liberate memory
    print('Blobs that are removed:', len(index_list))
    
    stack = from_folder_to_stack(stack_filename) # loading stack a second time
    stack = stack.view(dtype=np.float32) # Need to check if it is worth doing
    
    for i in range(len(index_list)):
        stack = set_to_int(stack, propsa[index_list[i]].coords, 0)
        
    if keep_label:
        for i in range(len(kept_labels)):
            stack = set_to_int(stack, propsa[kept_labels[i]].coords, kept_labels[i]) #cython
    
    print('End: individual "blobs" remaining', N-len(index_list))
    
    if keep_label:
        return np.array(stack,dtype=np.uint16)
    else:  
        return stack


def find_biggest_elements(stack_filename, min_threshold):
    
    label_stack = from_folder_to_stack(stack_filename) # loading stack a first time for connected-components labeling
    label_stack = label_stack.view(dtype=np.float32) # forces a specific size (should be native size after segmentation) (check if worth doing)
    label_stack[label_stack > 0] = 1 # binarization, checking below 0 should be unnecessary (if proper workflow order)
    label_stack, N = cc3d.connected_components(label_stack, out_dtype=np.uint32, return_N=True) # better than skimage ?
    #label_stack = measure.label(label_stack, background=0) #computationally expensive
    print('Individual "blobs":', N)
    
    propsa = measure.regionprops(label_stack)
    del label_stack #liberate memory
    
    index_list = []
    for count, item in enumerate(propsa):
        if len(item.coords) >= min_threshold:
            index_list.append(count)
            
    print(f'Blobs of size superior to {min_threshold}:', len(index_list))
                        
    return index_list, propsa

def keep_selected_elements(stack_filename, label_list):
    # Removing small elements increases the mean probability of remaining pixels. (Post-skeletonization)
    # Small elements should be associated with lower probabilities of axon presence (artifacts, blood vessels,...)
    # We want the "big" stuff
    
    label_stack = from_folder_to_stack(stack_filename) # loading stack a first time for connected-components labeling
    label_stack = label_stack.view(dtype=np.float32) # forces a specific size (should be native size after segmentation) (check if worth doing)
    label_stack[label_stack > 0] = 1 # binarization, checking below 0 should be unnecessary (if proper workflow order)
    label_stack, N = cc3d.connected_components(label_stack, out_dtype=np.uint32, return_N=True) # better than skimage ?
    #label_stack = measure.label(label_stack, background=0) #computationally expensive
    print('Start: individual "blobs":', N)
    
    propsa = measure.regionprops(label_stack)
    del label_stack #liberate memory
    
    stack = from_folder_to_stack(stack_filename) # loading stack a second time
    stack = stack.view(dtype=np.float32) # Need to check if it is worth doing
     
    if np.max(label_list) > len(propsa):
        print("Error: illegal label in input label_list. Stack unchanged ! ")
        return stack
        
    check_count = 0
    for count, item in enumerate(propsa):
        if count not in label_list:
            stack = set_to_int(stack, item.coords, 0) #cython
            check_count += 1
                
    print('End: individual "blobs" remaining', N-check_count)
                
    return stack

In [6]:
%%cython
import numpy as np
cimport numpy as np

def compute_sum(np.ndarray[float, ndim=3] vol, np.ndarray[long long, ndim=2] list_coords):
    cdef double sum_proba = 0.0
    cdef int i = 0
    for i in range(len(list_coords)):
        sum_proba += vol[list_coords[i][0],list_coords[i][1],list_coords[i][2]]
    
    return sum_proba/len(list_coords)

In [7]:
def compute_skel_probas(stack_filename, count_only = False):
    # Removing small elements increases the mean probability of remaining pixels. (Post-skeletonization)
    # Small elements should be associated with lower probabilities of axon presence (artifacts, blood vessels,...)
    # We want the "big" stuff
    
    label_stack = from_folder_to_stack(stack_filename) # loading stack a first time for connected-components labeling
    label_stack = label_stack.view(dtype=np.float32) # forces a specific size (should be native size after segmentation) (check if worth doing)
    label_stack[label_stack > 0] = 1 # binarization, checking below 0 should be unnecessary (if proper workflow order)
    label_stack, N = cc3d.connected_components(label_stack, out_dtype=np.uint32, return_N=True) # better than skimage ?
    #label_stack = measure.label(label_stack, background=0) #computationally expensive
    print('Start: individual "blobs":', N)
    
    propsa = measure.regionprops(label_stack)
    del label_stack #liberate memory
        
    stack = from_folder_to_stack(stack_filename) # loading stack a second time
    stack = stack.view(dtype=np.float32) # Need to check if it is worth doing
    probablity_densities = []
    voxel_count = 0
    for count, item in enumerate(propsa):
        if not count_only:
            probablity_densities.append(compute_sum(stack, item.coords))
        voxel_count += len(item.coords)
        
    print("Total number of voxels:", voxel_count)
                
    return probablity_densities

## CC3d only

In [5]:
dask_stack = dask_image.imread.imread(r'/data/Lucas/Substacks/AL207s/Al207-substack/*.tif')

In [6]:
labels_in = dask_stack.compute()

In [ ]:
cc3d.dust(labels_in,threshold=100)

In [2]:
#labels_in = np.ones((512, 512, 512), dtype=np.int32)
labels_in = np.random.randint(100, size=(64,64,64))
labels_in.astype(int)
cc3d.dust(labels_in,threshold=100) #dusting does not conserve the labels. Might be beneficial to use only the connected_components function. 
#labels_out, N = cc3d.connected_components(labels_in, out_dtype=np.uint32, return_N=True) #26 connectivity, max for uint32 is 4'294'967'296 (uint16 is not enough)

TypeError: No matching signature found

In [16]:
type(labels_in[2,2,2])
type(12)

int

In [ ]:
%%time 
#takes about 1 minute for (64,64,64) image

l = []

for label in range(1, N+1):
    extracted_image = labels_out*(labels_out == label)
    extracted_image = extracted_image[extracted_image!=0]
    l.append(len(extracted_image))

l=np.array(l)
print(np.max(l))

In [3]:
a = np.ones((12,12,12))
a[1,:,:] = 2

In [4]:
a = a[a!=1]

In [5]:
len(a)

144

## Testing Hybrid (skimage + cc3d)

In [6]:
%%time
stack = trim_small_elements(r'/data/Lucas/AL207/skel-seg-AL207_20220428_4/weighted_sum_skeleton', 2000)
from_vol_to_folder(stack, r'weighted_skeleton.tif', r'/data/Lucas/AL207/AL207_20220428_4_skel_trim')

Start: individual "blobs": 215100
Blobs that are removed: 214959
Stack dimensions: (1457, 2048, 2048)
/data/Lucas/AL207/AL207_20220428_4_skel_trim already exists. Will be overwritten.
CPU times: user 1min 38s, sys: 1min 43s, total: 3min 22s
Wall time: 5min 35s


In [7]:
%%time
stack = trim_small_elements(r'/data/Lucas/AL210/skel-seg-AL210_20220428_4/weighted_sum_skeleton', 2000)
from_vol_to_folder(stack, r'weighted_skeleton.tif', r'/data/Lucas/AL210/AL210_20220428_4_skel_trim')

Start: individual "blobs": 222144
Blobs that are removed: 222092
Stack dimensions: (1381, 2048, 2048)
/data/Lucas/AL210/AL210_20220428_4_skel_trim already exists. Will be overwritten.
CPU times: user 1min 46s, sys: 2min 48s, total: 4min 35s
Wall time: 7min 25s


In [33]:
%%time
stack = trim_small_elements(r'/data/Lucas/AL215/skel-seg-AL215_20220428_4/weighted_sum_skeleton', 1000000)
from_vol_to_folder(stack, r'weighted_skeleton.tif', r'/data/Lucas/AL215/AL215_20220428_4_skel_trim1e6')
del stack

Start: individual "blobs": 269868
Blobs that are removed: 269866
End: individual "blobs" remaining 2
Stack dimensions: (1359, 2048, 2048)
/data/Lucas/AL215/AL215_20220428_4_skel_trim1e6 already exists. Will be overwritten.
CPU times: user 1min 52s, sys: 2min 48s, total: 4min 41s
Wall time: 6min 50s


In [34]:
%%time
stack = trim_small_elements(r'/data/Lucas/AL257/skel-seg-AL257_20220428_4/weighted_sum_skeleton', 1000000)
from_vol_to_folder(stack, r'weighted_skeleton.tif', r'/data/Lucas/AL257/AL257_20220428_4_skel_trim1e6')
del stack

Start: individual "blobs": 273441
Blobs that are removed: 273440
End: individual "blobs" remaining 1
Stack dimensions: (1296, 2048, 2048)
/data/Lucas/AL257/AL257_20220428_4_skel_trim1e6 already exists. Will be overwritten.
CPU times: user 1min 24s, sys: 1min 26s, total: 2min 51s
Wall time: 4min 37s


## Analytics

## AL207 Dense

In [8]:
mouse = 'AL207'

In [10]:
i_list, prop = find_biggest_elements(r'/data/Lucas/AL207/skel-seg-AL207_dense/weighted_sum_skeleton', 1e4)

Individual "blobs": 73626
Blobs of size superior to 10000.0: 10


In [11]:
compute_skel_probas(f'/data/Lucas/AL207/skel-seg-AL207_dense/weighted_sum_skeleton', True)

Start: individual "blobs": 73626
Total number of voxels: 3023895


[]

In [12]:
print_biggest_connected_components(i_list, prop)

Index number: 617 size: 1594047
Index number: 5011 size: 46549
Index number: 25470 size: 33775
Index number: 29138 size: 12606
Index number: 31304 size: 27019
Index number: 38029 size: 19380
Index number: 39995 size: 211616
Index number: 44680 size: 152825
Index number: 54285 size: 14872
Index number: 61971 size: 10317


In [13]:
stack = trim_small_elements(r'/data/Lucas/AL207/skel-seg-AL207_dense/weighted_sum_skeleton', 1e4)
from_vol_to_folder(stack, r'weighted_skeleton.tif', r'/data/Lucas/AL207/AL207_dense_1e4')

Start: individual "blobs": 73626
Blobs that are removed: 73616
End: individual "blobs" remaining 10
New directory created at: /data/Lucas/AL207/AL207_dense_1e4
Stack dimensions: (1457, 2048, 2048)
/data/Lucas/AL207/AL207_dense_1e4 already exists. Will be overwritten.


In [14]:
stack = trim_small_elements(r'/data/Lucas/AL207/skel-seg-AL207_dense/weighted_sum_skeleton', 1e6)
from_vol_to_folder(stack, r'weighted_skeleton.tif', r'/data/Lucas/AL207/AL207_dense_1e6')

Start: individual "blobs": 73626
Blobs that are removed: 73625
End: individual "blobs" remaining 1
New directory created at: /data/Lucas/AL207/AL207_dense_1e6
Stack dimensions: (1457, 2048, 2048)
/data/Lucas/AL207/AL207_dense_1e6 already exists. Will be overwritten.


In [9]:
stack = trim_small_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_dense/weighted_sum_skeleton', 1e4, 1e6, True)
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_dense_skel_trim_1e4to1e6_kept')

Start: individual "blobs": 73626
Blobs that are removed: 73617
End: individual "blobs" remaining 9
New directory created at: /data/Lucas/AL207/AL207_dense_skel_trim_1e4to1e6_kept
Stack dimensions: (1457, 2048, 2048)
/data/Lucas/AL207/AL207_dense_skel_trim_1e4to1e6_kept already exists. Will be overwritten.


In [11]:
stack = keep_selected_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_dense/weighted_sum_skeleton', [617, 31304, 39995, 54285, 44680])
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_dense_skel_fine_trim')

Start: individual "blobs": 73626
End: individual "blobs" remaining 5
New directory created at: /data/Lucas/AL207/AL207_dense_skel_fine_trim
Stack dimensions: (1457, 2048, 2048)
/data/Lucas/AL207/AL207_dense_skel_fine_trim already exists. Will be overwritten.


In [15]:
del stack

## AL210

In [8]:
mouse = 'AL210'

In [7]:
#AL210
i_list, prop = find_biggest_elements(r'/data/Lucas/AL210/skel-seg-AL210_20220506_2/weighted_sum_skeleton', 10000)

Individual "blobs": 188572
Blobs of size superior to 10000: 6


In [8]:
print_biggest_connected_components(i_list, prop)

Index number: 12 size: 15106696
Index number: 24261 size: 14664
Index number: 49292 size: 11429
Index number: 158793 size: 17749
Index number: 162731 size: 14215
Index number: 176515 size: 15601


In [9]:
stack = trim_small_elements(r'/data/Lucas/AL210/skel-seg-AL210_20220506_2/weighted_sum_skeleton', 1e4, 1e5)
from_vol_to_folder(stack, r'weighted_skeleton.tif', r'/data/Lucas/AL210/AL210_20220506_2_skel_trim_1e4to1e5')

Start: individual "blobs": 188572
Blobs that are removed: 188567
End: individual "blobs" remaining 5
New directory created at: /data/Lucas/AL210/AL210_20220506_2_skel_trim_1e4to1e5
Stack dimensions: (1381, 2048, 2048)
/data/Lucas/AL210/AL210_20220506_2_skel_trim_1e4to1e5 already exists. Will be overwritten.


In [9]:
stack = trim_small_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', 1e4, 1e5, True)
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_trim_1e4to1e5_kept')

Start: individual "blobs": 188572
Blobs that are removed: 188567
End: individual "blobs" remaining 5
New directory created at: /data/Lucas/AL210/AL210_20220506_2_skel_trim_1e4to1e5_kept
Stack dimensions: (1381, 2048, 2048)
/data/Lucas/AL210/AL210_20220506_2_skel_trim_1e4to1e5_kept already exists. Will be overwritten.


In [15]:
stack = trim_small_elements(r'/data/Lucas/AL210/skel-seg-AL210_20220506_2/weighted_sum_skeleton', 1e6)
from_vol_to_folder(stack, r'weighted_skeleton.tif', r'/data/Lucas/AL210/AL210_20220506_2_skel_trim_1e6')

Start: individual "blobs": 188572
Blobs that are removed: 188571
End: individual "blobs" remaining 1
Stack dimensions: (1381, 2048, 2048)
/data/Lucas/AL210/AL210_20220506_2_skel_trim_1e6 already exists. Will be overwritten.


In [10]:
stack = keep_selected_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', [12, 162731])
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_fine_trim')

Start: individual "blobs": 188572
End: individual "blobs" remaining 2
New directory created at: /data/Lucas/AL210/AL210_20220506_2_skel_fine_trim
Stack dimensions: (1381, 2048, 2048)
/data/Lucas/AL210/AL210_20220506_2_skel_fine_trim already exists. Will be overwritten.


In [1]:
del prop

NameError: name 'prop' is not defined

## AL207

In [9]:
mouse = 'AL207'

In [16]:
i_list, prop = find_biggest_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', 1e4)

Individual "blobs": 178285
Blobs of size superior to 10000.0: 14


In [17]:
print_biggest_connected_components(i_list, prop)

Index number: 1110 size: 3652024
Index number: 5089 size: 44222
Index number: 8335 size: 12154
Index number: 18790 size: 14606
Index number: 20044 size: 49931
Index number: 23984 size: 11978
Index number: 82169 size: 32360
Index number: 83584 size: 10596
Index number: 85569 size: 32062
Index number: 99631 size: 13818
Index number: 109569 size: 22464
Index number: 137546 size: 12628
Index number: 138428 size: 14537
Index number: 158562 size: 28891


In [18]:
compute_skel_probas(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', True)

Start: individual "blobs": 178285
Total number of voxels: 5871383


[]

In [20]:
stack = trim_small_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', 1e6)
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_trim_1e6')

Start: individual "blobs": 178285
Blobs that are removed: 178284
End: individual "blobs" remaining 1
New directory created at: /data/Lucas/AL207/AL207_20220506_2_skel_trim_1e6
Stack dimensions: (1457, 2048, 2048)
/data/Lucas/AL207/AL207_20220506_2_skel_trim_1e6 already exists. Will be overwritten.


In [18]:
stack = trim_small_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', 1e4, 1e5, True)
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_trim_1e4to1e5_kept')

Start: individual "blobs": 178285
Blobs that are removed: 178272
End: individual "blobs" remaining 13
Stack dimensions: (1457, 2048, 2048)
/data/Lucas/AL207/AL207_20220506_2_skel_trim_1e4to1e5_kept already exists. Will be overwritten.


In [ ]:
proba = compute_skel_probas(f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_trim_1e4to1e5')

In [10]:
proba = compute_skel_probas(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', True)

Start: individual "blobs": 178285
Total number of voxels: 5871383


In [11]:
print(proba)

[]


In [ ]:
stack = keep_selected_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', [1110, 18790])
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_fine_trim')

Start: individual "blobs": 178285


### Default weights AL207

In [14]:
mouse = 'AL207'

In [15]:
i_list, prop = find_biggest_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_default/weighted_sum_skeleton', 1e4)
print_biggest_connected_components(i_list, prop)

Individual "blobs": 36131
Blobs of size superior to 10000.0: 6
Index number: 242 size: 1150423
Index number: 2591 size: 18163
Index number: 14382 size: 20682
Index number: 20776 size: 14794
Index number: 22742 size: 178131
Index number: 23948 size: 130759


In [16]:
compute_skel_probas(f'/data/Lucas/{mouse}/skel-seg-{mouse}_default/weighted_sum_skeleton', True)

Start: individual "blobs": 36131
Total number of voxels: 2016841


[]

In [12]:
stack = trim_small_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_default/weighted_sum_skeleton', 1e4, 1e6, True)
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_default_skel_trim_1e4to1e6_kept')

Start: individual "blobs": 36131
Blobs that are removed: 36126
End: individual "blobs" remaining 5
New directory created at: /data/Lucas/AL207/AL207_default_skel_trim_1e4to1e6_kept
Stack dimensions: (1457, 2048, 2048)
/data/Lucas/AL207/AL207_default_skel_trim_1e4to1e6_kept already exists. Will be overwritten.


In [13]:
stack = trim_small_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_default/weighted_sum_skeleton', 1e6)
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_default_skel_trim_1e6')

Start: individual "blobs": 36131
Blobs that are removed: 36130
End: individual "blobs" remaining 1
New directory created at: /data/Lucas/AL207/AL207_default_skel_trim_1e6
Stack dimensions: (1457, 2048, 2048)
/data/Lucas/AL207/AL207_default_skel_trim_1e6 already exists. Will be overwritten.


In [14]:
stack = trim_small_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_default/weighted_sum_skeleton', 1e4)
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_default_skel_trim_1e4')

Start: individual "blobs": 36131
Blobs that are removed: 36125
End: individual "blobs" remaining 6
New directory created at: /data/Lucas/AL207/AL207_default_skel_trim_1e4
Stack dimensions: (1457, 2048, 2048)
/data/Lucas/AL207/AL207_default_skel_trim_1e4 already exists. Will be overwritten.


In [15]:
stack = keep_selected_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_dense/weighted_sum_skeleton', [242, 22742, 23948, 20776])
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_dense_skel_fine_trim')

Start: individual "blobs": 73626
End: individual "blobs" remaining 4
Stack dimensions: (1457, 2048, 2048)
/data/Lucas/AL207/AL207_dense_skel_fine_trim already exists. Will be overwritten.


## AL213

In [14]:
mouse = 'AL213'

In [8]:
i_list, prop = find_biggest_elements(r'/data/Lucas/AL213/skel-seg-AL213_20220506_2/weighted_sum_skeleton', 10000)
print_biggest_connected_components(i_list, prop)

Individual "blobs": 236119
Blobs of size superior to 10000: 5
Index number: 138 size: 22812456
Index number: 11914 size: 11272
Index number: 138417 size: 26212
Index number: 147973 size: 32862
Index number: 178435 size: 15255


In [9]:
stack = trim_small_elements(r'/data/Lucas/AL213/skel-seg-AL213_20220506_2/weighted_sum_skeleton', 1e4, 1e5)
from_vol_to_folder(stack, r'weighted_skeleton.tif', r'/data/Lucas/AL213/AL213_20220506_2_skel_trim_1e4to1e5')

Start: individual "blobs": 236119
Blobs that are removed: 236115
End: individual "blobs" remaining 4
New directory created at: /data/Lucas/AL213/AL213_20220506_2_skel_trim_1e4to1e5
Stack dimensions: (1537, 2048, 2048)
/data/Lucas/AL213/AL213_20220506_2_skel_trim_1e4to1e5 already exists. Will be overwritten.


In [12]:
stack = trim_small_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', 1e4, 1e5, True)
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_trim_1e4to1e5_kept')

Start: individual "blobs": 236119
Blobs that are removed: 236115
End: individual "blobs" remaining 4
New directory created at: /data/Lucas/AL213/AL213_20220506_2_skel_trim_1e4to1e5_kept
Stack dimensions: (1537, 2048, 2048)
/data/Lucas/AL213/AL213_20220506_2_skel_trim_1e4to1e5_kept already exists. Will be overwritten.


In [10]:
stack = trim_small_elements(r'/data/Lucas/AL213/skel-seg-AL213_20220506_2/weighted_sum_skeleton', 1e6)
from_vol_to_folder(stack, r'weighted_skeleton.tif', r'/data/Lucas/AL213/AL213_20220506_2_skel_trim_1e6')

Start: individual "blobs": 236119
Blobs that are removed: 236118
End: individual "blobs" remaining 1
Stack dimensions: (1537, 2048, 2048)
/data/Lucas/AL213/AL213_20220506_2_skel_trim_1e6 already exists. Will be overwritten.


In [15]:
stack = trim_small_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', 1e4)
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_trim_1e4')

Start: individual "blobs": 236119
Blobs that are removed: 236114
End: individual "blobs" remaining 5
New directory created at: /data/Lucas/AL213/AL213_20220506_2_skel_trim_1e4
Stack dimensions: (1537, 2048, 2048)
/data/Lucas/AL213/AL213_20220506_2_skel_trim_1e4 already exists. Will be overwritten.


In [13]:
del stack

## AL209

In [ ]:
mouse = 'AL209'

In [ ]:
stack = trim_small_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', 1e6)
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_trim_1e6')

In [12]:
i_list, prop = find_biggest_elements(r'/data/Lucas/AL209/skel-seg-AL209_20220506_2/weighted_sum_skeleton', 10000)
print_biggest_connected_components(i_list, prop)

Individual "blobs": 165638
Blobs of size superior to 10000: 11
Index number: 2 size: 32382
Index number: 59 size: 4162406
Index number: 82981 size: 14176
Index number: 85611 size: 34287
Index number: 105259 size: 181854
Index number: 113582 size: 16443
Index number: 126873 size: 20799
Index number: 128635 size: 16223
Index number: 129409 size: 10219
Index number: 131573 size: 10081
Index number: 142982 size: 17374


In [7]:
stack = trim_small_elements(r'/data/Lucas/AL209/skel-seg-AL209_20220506_2/weighted_sum_skeleton', 1e4)
from_vol_to_folder(stack, r'weighted_skeleton.tif', r'/data/Lucas/AL209/AL209_20220506_2_skel_trim_1e4')

Start: individual "blobs": 165638
Blobs that are removed: 165627
End: individual "blobs" remaining 11
New directory created at: /data/Lucas/AL209/AL209_20220506_2_skel_trim_1e4
Stack dimensions: (1702, 2048, 2048)
/data/Lucas/AL209/AL209_20220506_2_skel_trim_1e4 already exists. Will be overwritten.


In [14]:
stack = trim_small_elements(r'/data/Lucas/AL209/skel-seg-AL209_20220506_2/weighted_sum_skeleton', 1e5, 1e6)
from_vol_to_folder(stack, r'weighted_skeleton.tif', r'/data/Lucas/AL209/AL209_20220506_2_skel_trim_1e5to1e6')

Start: individual "blobs": 165638
Blobs that are removed: 165637
End: individual "blobs" remaining 1
New directory created at: /data/Lucas/AL209/AL209_20220506_2_skel_trim_1e5to1e6
Stack dimensions: (1702, 2048, 2048)
/data/Lucas/AL209/AL209_20220506_2_skel_trim_1e5to1e6 already exists. Will be overwritten.


In [8]:
stack = trim_small_elements(r'/data/Lucas/AL209/skel-seg-AL209_20220506_2/weighted_sum_skeleton', 1e4, 1e5)
from_vol_to_folder(stack, r'weighted_skeleton.tif', r'/data/Lucas/AL209/AL209_20220506_2_skel_trim_1e6')

Start: individual "blobs": 165638
Blobs that are removed: 165629
End: individual "blobs" remaining 9
Stack dimensions: (1702, 2048, 2048)
/data/Lucas/AL209/AL209_20220506_2_skel_trim_1e6 already exists. Will be overwritten.


In [9]:
stack = trim_small_elements(r'/data/Lucas/AL209/skel-seg-AL209_20220506_2/weighted_sum_skeleton', 1e5)
from_vol_to_folder(stack, r'weighted_skeleton.tif', r'/data/Lucas/AL209/AL209_20220506_2_skel_trim_1e5')

Start: individual "blobs": 165638
Blobs that are removed: 165636
End: individual "blobs" remaining 2
New directory created at: /data/Lucas/AL209/AL209_20220506_2_skel_trim_1e5
Stack dimensions: (1702, 2048, 2048)
/data/Lucas/AL209/AL209_20220506_2_skel_trim_1e5 already exists. Will be overwritten.


## AL215

In [11]:
mouse = 'AL215'

In [12]:
compute_skel_probas(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', True)

Start: individual "blobs": 250430
Total number of voxels: 24812069


[]

In [15]:
i_list, prop = find_biggest_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', 1e4)
print_biggest_connected_components(i_list, prop)

Individual "blobs": 250430
Blobs of size superior to 10000.0: 11
Index number: 6 size: 19632984
Index number: 3766 size: 11522
Index number: 13631 size: 25157
Index number: 21192 size: 2732424
Index number: 26906 size: 26341
Index number: 124828 size: 12278
Index number: 156659 size: 67780
Index number: 157238 size: 13845
Index number: 186140 size: 10015
Index number: 211523 size: 12148
Index number: 215752 size: 36056


In [19]:
stack = trim_small_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', 1e4, 1e6, True)
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_trim_1e4to1e6_kept')

Start: individual "blobs": 250430
Blobs that are removed: 250421
End: individual "blobs" remaining 9
New directory created at: /data/Lucas/AL215/AL215_20220506_2_skel_trim_1e4to1e6_kept
Stack dimensions: (1359, 2048, 2048)
/data/Lucas/AL215/AL215_20220506_2_skel_trim_1e4to1e6_kept already exists. Will be overwritten.


In [20]:
stack = trim_small_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', 1e6)
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_trim_1e6_kept')

Start: individual "blobs": 250430
Blobs that are removed: 250428
End: individual "blobs" remaining 2
New directory created at: /data/Lucas/AL215/AL215_20220506_2_skel_trim_1e6_kept
Stack dimensions: (1359, 2048, 2048)
/data/Lucas/AL215/AL215_20220506_2_skel_trim_1e6_kept already exists. Will be overwritten.


In [21]:
stack = trim_small_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', 1e4)
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_trim_1e4')

Start: individual "blobs": 250430
Blobs that are removed: 250419
End: individual "blobs" remaining 11
New directory created at: /data/Lucas/AL215/AL215_20220506_2_skel_trim_1e4
Stack dimensions: (1359, 2048, 2048)
/data/Lucas/AL215/AL215_20220506_2_skel_trim_1e4 already exists. Will be overwritten.


In [22]:
stack = trim_small_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', 1e7)
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_trim_1e7')

Start: individual "blobs": 250430
Blobs that are removed: 250429
End: individual "blobs" remaining 1
New directory created at: /data/Lucas/AL215/AL215_20220506_2_skel_trim_1e7
Stack dimensions: (1359, 2048, 2048)
/data/Lucas/AL215/AL215_20220506_2_skel_trim_1e7 already exists. Will be overwritten.


In [24]:
stack = trim_small_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', 1e6, 1e7)
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_trim_1e6to1e7')

Start: individual "blobs": 250430
Blobs that are removed: 250429
End: individual "blobs" remaining 1
Stack dimensions: (1359, 2048, 2048)
/data/Lucas/AL215/AL215_20220506_2_skel_trim_1e6to1e7 already exists. Will be overwritten.


In [16]:
stack = keep_selected_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', [6, 21192, 211523])
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_fine_trim')

Start: individual "blobs": 250430
End: individual "blobs" remaining 3
New directory created at: /data/Lucas/AL215/AL215_20220506_2_skel_fine_trim
Stack dimensions: (1359, 2048, 2048)
/data/Lucas/AL215/AL215_20220506_2_skel_fine_trim already exists. Will be overwritten.


## AL246

In [8]:
mouse = 'AL246'

In [9]:
i_list, prop = find_biggest_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', 1e4)
print_biggest_connected_components(i_list, prop)

Individual "blobs": 305632
Blobs of size superior to 10000.0: 17
Index number: 12 size: 7732296
Index number: 1292 size: 49687
Index number: 13994 size: 10997
Index number: 35514 size: 40956
Index number: 70976 size: 13783
Index number: 71413 size: 11508
Index number: 85431 size: 18248
Index number: 105729 size: 24154
Index number: 106877 size: 11761
Index number: 108069 size: 38800
Index number: 145746 size: 10201
Index number: 202012 size: 17416
Index number: 225843 size: 34220
Index number: 232931 size: 31859
Index number: 239488 size: 15035
Index number: 266344 size: 10308
Index number: 298449 size: 14211


In [10]:
compute_skel_probas(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', True)

Start: individual "blobs": 305632
Total number of voxels: 10591606


[]

In [11]:
stack = trim_small_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', 1e4, 1e6, True)
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_trim_1e4to1e6_kept')

Start: individual "blobs": 305632
Blobs that are removed: 305616
End: individual "blobs" remaining 16
New directory created at: /data/Lucas/AL246/AL246_20220506_2_skel_trim_1e4to1e6_kept
Stack dimensions: (1500, 2048, 4096)
/data/Lucas/AL246/AL246_20220506_2_skel_trim_1e4to1e6_kept already exists. Will be overwritten.


In [12]:
stack = trim_small_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', 1e6)
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_trim_1e6')

Start: individual "blobs": 305632
Blobs that are removed: 305631
End: individual "blobs" remaining 1
New directory created at: /data/Lucas/AL246/AL246_20220506_2_skel_trim_1e6
Stack dimensions: (1500, 2048, 2048)
/data/Lucas/AL246/AL246_20220506_2_skel_trim_1e6 already exists. Will be overwritten.


In [13]:
stack = trim_small_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', 1e4)
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_trim_1e4')

Start: individual "blobs": 305632
Blobs that are removed: 305615
End: individual "blobs" remaining 17
New directory created at: /data/Lucas/AL246/AL246_20220506_2_skel_trim_1e4
Stack dimensions: (1500, 2048, 2048)
/data/Lucas/AL246/AL246_20220506_2_skel_trim_1e4 already exists. Will be overwritten.


## AL257

In [9]:
mouse = 'AL257'

In [22]:
i_list, prop = find_biggest_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', 1e4)
print_biggest_connected_components(i_list, prop)

Individual "blobs": 315179
Blobs of size superior to 10000.0: 4
Index number: 1127 size: 10319261
Index number: 215010 size: 14163
Index number: 246460 size: 20105
Index number: 290415 size: 12062


#### I kept only the biggest one, the other 3 are artifacts. 

In [20]:
compute_skel_probas(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', True)

Start: individual "blobs": 315179
Total number of voxels: 12396275


[]

In [23]:
stack = trim_small_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', 1e4)
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_trim_1e4')

Start: individual "blobs": 315179
Blobs that are removed: 315175
End: individual "blobs" remaining 4
Stack dimensions: (1296, 2048, 2048)
/data/Lucas/AL257/AL257_20220506_2_skel_trim_1e4 already exists. Will be overwritten.


In [10]:
stack = trim_small_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', 1e4, 1e6, True)
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_trim_1e4to1e6_kept')

Start: individual "blobs": 315179
Blobs that are removed: 315176
End: individual "blobs" remaining 3
Stack dimensions: (1296, 2048, 2048)
/data/Lucas/AL257/AL257_20220506_2_skel_trim_1e4to1e6_kept already exists. Will be overwritten.


In [24]:
stack = trim_small_elements(f'/data/Lucas/{mouse}/skel-seg-{mouse}_20220506_2/weighted_sum_skeleton', 1e6)
from_vol_to_folder(stack, r'weighted_skeleton.tif', f'/data/Lucas/{mouse}/{mouse}_20220506_2_skel_trim_1e6')

Start: individual "blobs": 315179
Blobs that are removed: 315178
End: individual "blobs" remaining 1
New directory created at: /data/Lucas/AL257/AL257_20220506_2_skel_trim_1e6
Stack dimensions: (1296, 2048, 2048)
/data/Lucas/AL257/AL257_20220506_2_skel_trim_1e6 already exists. Will be overwritten.


In [10]:
i_list, _ = find_biggest_elements(r'/data/Lucas/AL257/skel-seg-AL257_20220428_4/weighted_sum_skeleton', 10000)

Individual "blobs": 273441
Blobs of size superior to 10000: 15


In [6]:
def print_biggest_connected_components(indexes, propsa):
    max_size = 0
    for index in indexes:
        ccompo_size = len(propsa[index].coords)
        print(index, ccompo_size)

## Old AL215

In [7]:
i_list, prop = find_biggest_elements(r'/data/Lucas/AL215/skel-seg-AL215_20220428_4/weighted_sum_skeleton', 10000)

Individual "blobs": 269868
Blobs of size superior to 10000: 18


In [31]:
print_biggest_connected_components(i_list, prop)

8 24707858
13896 41908
17630 16118
21599 2975779
21612 10030
53831 16411
77573 10579
126159 15876
128825 13973
133647 34214
142462 69692
167920 11422
176482 13356
191697 15502
199147 15136
226928 18326
233509 36058
261815 27248


In [ ]:
%%time
stack = trim_small_elements(r'/data/Lucas/AL215/skel-seg-AL215_20220428_4/weighted_sum_skeleton', 6*1e4, 1e5)
from_vol_to_folder(stack, r'weighted_skeleton.tif', r'/data/Lucas/AL215/AL215_20220428_4_skel_trim_split2nd')
del stack

Start: individual "blobs": 269868


In [37]:
%%time
stack = trim_small_elements(r'/data/Lucas/AL215/skel-seg-AL215_20220428_4/weighted_sum_skeleton', 1e7)
from_vol_to_folder(stack, r'weighted_skeleton.tif', r'/data/Lucas/AL215/AL215_20220428_4_skel_trim_split1st')
del stack

Start: individual "blobs": 269868
Blobs that are removed: 269867
End: individual "blobs" remaining 1
Stack dimensions: (1359, 2048, 2048)
/data/Lucas/AL215/AL215_20220428_4_skel_trim_split1st already exists. Will be overwritten.
CPU times: user 1min 47s, sys: 2min 2s, total: 3min 50s
Wall time: 5min 57s


In [ ]:
i_list, _ = find_biggest_elements(r'/data/Lucas/AL257/skel-seg-AL257_20220428_4/weighted_sum_skeleton', 10000)

## Tree-metrics or repeatability

1) First element to consider is repeatability of the pipeline: idea that quantification should give similar results in different brains. This part is most important. 
2) Tree-comparision might be interessant to consider difference in projection patterns between different mice. Assure a better performance of the model maybe. 

In [8]:
tree1 = prop[i_list[2]].coords

In [9]:
tree1

array([[ 130, 1346,  910],
       [ 131, 1342,  908],
       [ 131, 1343,  910],
       ...,
       [ 236, 1266,  935],
       [ 237, 1267,  934],
       [ 237, 1268,  934]])

In [12]:
%%cython 
import numpy as np
cimport numpy as np
from tqdm import tqdm

def maximum_euclidean_dist(np.ndarray[long long, ndim=2] arr):
    cdef float max_dist = 0
    cdef int coo1_max = 0
    cdef int coo2_max = 0
    for i in tqdm(range(len(arr))): 
        for j in range(i+1, len(arr)):
            dist = np.linalg.norm(arr[i] - arr[j])
            if dist > max_dist:
                max_dist = dist
                coo1_max = i
                coo2_max = j
    return max_dist, [coo1_max, coo2_max]

In file included from /home/lucasdelez/anaconda3/envs/data_analysis/lib/python3.9/site-packages/numpy/core/include/numpy/ndarraytypes.h:1969,
                 from /home/lucasdelez/anaconda3/envs/data_analysis/lib/python3.9/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/lucasdelez/anaconda3/envs/data_analysis/lib/python3.9/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /home/lucasdelez/.cache/ipython/cython/_cython_magic_5ff94cd396ab465949e4843cd0af9004.c:710:
/home/lucasdelez/anaconda3/envs/data_analysis/lib/python3.9/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~


In [13]:
maximum_euclidean_dist(tree1)

100%|█████████████████████████████████████| 16118/16118 [07:59<00:00, 33.61it/s]


(195.67575073242188, [304, 11833])

In [53]:
np.linalg.norm(tree1[0]-tree1[2])

3.1622776601683795

In [58]:
type(tree1)

numpy.ndarray

In [ ]:
%%cython 
import numpy as np
cimport numpy as np
from tqdm import tqdm

### Faster way to compute this but should consider both coordinates in optimization.

def maximum_euclidean_dist_connected(np.ndarray[long long, ndim=2] arr, long long step):
    cdef float max_dist = 0
    cdef int coo1_max = 0
    cdef int coo2_max = 0
    
    list = []
    my_range = range(0, len(arr), step)
    range_list = []
    range_list.append(my_range)
    while step > 10:
        for r in range_list:
            for i in r:
                for j in range(len(arr)):
                    dist = np.linalg.norm(arr[i]-arr[j])
                    if dist > max_dist:
                        max_dist = dist
                        coo1_max = i
                        coo2_max = j
                    if dist + step > max_dist and dist - step < max_dist:
                        list.append([i, j])
                    if dist > max_dist + step:
                        list = []
        step = step//10
        range_list = []
        for i in range(len(list))
            range_list.append(range(
    
    for i in tqdm(range(len(arr))): 
        for j in range(i+1, len(arr)):
            dist = np.linalg.norm(arr[i] - arr[j])
            if dist > max_dist:
                max_dist = dist
                coo1_max = i
                coo2_max = j
    return max_dist, [coo1_max, coo2_max]